### Hierarchical Architecture for Linear Regression Updates Gossip

#### Create a Node Class

Each Node (Machine) has a left child, right child, address, and a parent. <br>This architecture could later be extended to incorporate multiple children for each node (k-children).

In [137]:
class Node:
    def __init__(self):
        self.left = None
        self.right = None
        self.parent = None
        self.addr = None

#### Create the Hierarchy

Since any Machine can be used as the main Master (root), we will use a divide-and-conquer approach to create the hierarchy.

In [138]:
def build_tree(nodes):
    if (len(nodes) == 0):
        return None
    mid = len(nodes)/2
    root = Node()
    root.addr = nodes[mid]
    root.left = buildTree(nodes[:mid])
    root.right = buildTree(nodes[mid+1:])
    return root

Setup the Parents for each Machine. This will increase efficiency while sending messages up the Tree

In [139]:
def get_parent(const_root, curr_node):
    if (const_root is None):
        return None
    if (const_root == curr_node):
        return const_root
    if (const_root.left == curr_node or const_root.right == curr_node):
        return const_root
    parent_on_left = get_parent(const_root.left, curr_node) 
    return parent_on_left if parent_on_left != None else get_parent(const_root.right, curr_node)

def setup_parents(const_root=root, curr_node=root):
    if (curr_node != None):
        curr_node.parent = get_parent(const_root=const_root, curr_node=curr_node)
        setup_parents(const_root=root, curr_node=curr_node.left)
        setup_parents(const_root=root, curr_node=curr_node.right)

#### Send Updates in Both Directions

In [140]:
def send_update_down(src_machine, target_machine, update):
    if (src_machine is None):
        print ("Trying to route through other subtree or Failed")
        return False
    elif (src_machine == target_machine):
        print (target_machine.addr + " received " + update)
        return True
    else:
        sent_through_left = send_update_down(src_machine.left, target_machine, update)
        return sent_through_left if sent_through_left != False else send_update_down(src_machine.right, target_machine, update)

def send_update_up(src_machine, target_machine, update):
    while (src_machine != None and src_machine != target_machine):
        src_machine = src_machine.parent
    if (src_machine is None):
        print ("Cannot Send Update")
    else:
        print (target_machine.addr + " received " + update)

In [141]:
nodes = ['1', '2', '3', '4', '5', '6', '7']
root = build_tree(nodes)
setup_parents(curr_node=root)